In [1]:
#https://dev.pokemontcg.io/dashboard
#https://github.com/PokemonTCG/pokemon-tcg-sdk-python

In [1]:
API_KEY = "1321d2cc-d928-4480-8d30-4e355eebc3aa"

In [4]:
!pip install pokemontcgsdk

You should consider upgrading via the 'c:\users\maxpr\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [5]:
from pokemontcgsdk import RestClient

RestClient.configure(API_KEY)

In [6]:
from pokemontcgsdk import Card
cards = Card.where(q='set.name:generations supertype:pokemon')

In [10]:
single_card = cards[0]

In [16]:
single_card.__dict__

{'abilities': None,
 'artist': 'Eske Yoshinob',
 'ancientTrait': None,
 'attacks': [Attack(name='Frog Hop', cost=['Grass', 'Colorless', 'Colorless'], convertedEnergyCost=3, damage='40+', text='Flip a coin. If heads, this attack does 40 more damage.'),
  Attack(name='Poison Impact', cost=['Grass', 'Grass', 'Colorless', 'Colorless'], convertedEnergyCost=4, damage='80', text="Your opponent's Active Pokémon is now Asleep and Poisoned.")],
 'cardmarket': Cardmarket(url='https://prices.pokemontcg.io/cardmarket/g1-1', updatedAt='2022/02/11', prices=CardmarketPrices(averageSellPrice=6.86, lowPrice=2.99, trendPrice=5.93, germanProLow=None, suggestedPrice=None, reverseHoloSell=None, reverseHoloLow=None, reverseHoloTrend=6.21, lowPriceExPlus=4.49, avg1=6.0, avg7=6.85, avg30=7.0, reverseHoloAvg1=6.99, reverseHoloAvg7=5.52, reverseHoloAvg30=5.03)),
 'convertedRetreatCost': 4,
 'evolvesFrom': None,
 'flavorText': None,
 'hp': '180',
 'id': 'g1-1',
 'images': CardImage(small='https://images.pokemontc

In [20]:
print(single_card.name)
print(single_card.nationalPokedexNumbers)
print(single_card.subtypes)
print(single_card.rarity)

Venusaur-EX
[3]
['Basic', 'EX']
Rare Holo EX


## For japanese

In [2]:
base_url = "https://jp.pokellector.com/sets"

In [3]:
!pip install webdriver-manager selenium wget tqdm

You should consider upgrading via the 'c:\users\maxpr\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [4]:
from selenium import webdriver
import time
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import wget
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from tqdm import tqdm_notebook

In [5]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downloads\\chromedriver_win32\\chromedriver.exe" )
driver.implicitly_wait(10)
data = []

C:\Users\maxpr\AppData\Local\Temp/ipykernel_22244/3374932004.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downloads\\chromedriver_win32\\chromedriver.exe" )


In [6]:
driver.get(base_url)

In [7]:
series = driver.find_elements_by_class_name('set')
blocks = driver.find_elements_by_class_name('buttonlisting')

C:\Users\maxpr\AppData\Local\Temp/ipykernel_22244/2139438567.py:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  series = driver.find_elements_by_class_name('set')
C:\Users\maxpr\AppData\Local\Temp/ipykernel_22244/2139438567.py:2: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  blocks = driver.find_elements_by_class_name('buttonlisting')


In [138]:
data_blocks = []
for idx, block in enumerate(blocks):
    print(series[idx].text)
    #print(ext.text)
    #ext_name = 
    for extension in block.find_elements_by_class_name('button'):
        ext_code = extension.get_attribute('name')
        ext_url = extension.get_attribute('href')
        ext_name = extension.get_attribute('title')
        ext_image_url = extension.find_element_by_css_selector('img').get_attribute('src')
        print(ext_code)
        print(ext_url)
        print(ext_name)

        data_blocks.append([series[idx].text, ext_code, ext_url, ext_name, ext_image_url])
df_ext = pd.DataFrame(data_blocks, columns =["serie", "ext_code", "ext_url", "ext_name", "ext_img_url"])

Sword & Shield Series
S9A
https://jp.pokellector.com/sets/S9A-Battle-Region
Battle Region Set
S9
https://jp.pokellector.com/sets/S9-Star-Birth
Star Birth Set
SI
https://jp.pokellector.com/sets/SI-Start-Deck-100
Start Deck 100 Set
S8B
https://jp.pokellector.com/sets/S8B-VMAX-Climax


C:\Users\maxpr\AppData\Local\Programs\Python\Python38\lib\site-packages\selenium\webdriver\remote\webelement.py:464: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
C:\Users\maxpr\AppData\Local\Programs\Python\Python38\lib\site-packages\selenium\webdriver\remote\webelement.py:484: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


VMAX Climax Set
S8AP
https://jp.pokellector.com/sets/S8AP-25th-Anniversary-Promo-Pack
25th Anniversary Promo Pack Set
S8A
https://jp.pokellector.com/sets/S8A-25th-Anniversary-Collection
25th Anniversary Collection Set
S8
https://jp.pokellector.com/sets/S8-Fusion-ARTS
Fusion ARTS Set
S7R
https://jp.pokellector.com/sets/S7R-Blue-Sky-Stream
Blue Sky Stream Set
S7D
https://jp.pokellector.com/sets/S7D-Towering-Perfection
Towering Perfection Set
S6A
https://jp.pokellector.com/sets/S6A-Eevee-Heroes
Eevee Heroes Set
S6K
https://jp.pokellector.com/sets/S6K-Jet-Black-Spirit
Jet Black Spirit Set
S6H
https://jp.pokellector.com/sets/S6H-Silver-Lance
Silver Lance Set
S5A
https://jp.pokellector.com/sets/S5A-Matchless-Fighter
Matchless Fighter Set
S5R
https://jp.pokellector.com/sets/S5R-Rapid-Strike-Master
Rapid Strike Master Set
S5I
https://jp.pokellector.com/sets/S5I-Single-Strike-Master
Single Strike Master Set
S4A
https://jp.pokellector.com/sets/S4A-Shiny-Star-V
Shiny Star V Set
S4
https://jp.poke

In [139]:
df_ext

,serie,ext_code,ext_url,ext_name,ext_img_url
0,Sword & Shield Series,S9A,https://jp.pokellector.com/sets/S9A-Battle-Region,Battle Region Set,https://den-media.pokellector.com/logos/Battle...
1,Sword & Shield Series,S9,https://jp.pokellector.com/sets/S9-Star-Birth,Star Birth Set,https://den-media.pokellector.com/logos/Star-B...
2,Sword & Shield Series,SI,https://jp.pokellector.com/sets/SI-Start-Deck-100,Start Deck 100 Set,https://den-media.pokellector.com/logos/Start-...
3,Sword & Shield Series,S8B,https://jp.pokellector.com/sets/S8B-VMAX-Climax,VMAX Climax Set,https://den-media.pokellector.com/logos/VMAX-C...
4,Sword & Shield Series,S8AP,https://jp.pokellector.com/sets/S8AP-25th-Anni...,25th Anniversary Promo Pack Set,https://den-media.pokellector.com/logos/25th-A...
...,...,...,...,...,...
151,Original Series,B05,https://jp.pokellector.com/sets/B05-Leaders-St...,Leader's Stadium Set,https://den-media.pokellector.com/logos/Leader...
152,Original Series,B04,https://jp.pokellector.com/sets/B04-Rocket-Gang,Rocket Gang Set,https://den-media.pokellector.com/logos/Rocket...
153,Original Series,B03,https://jp.pokellector.com/sets/B03-Mystery-of...,Mystery of the Fossils Set,https://den-media.pokellector.com/logos/Myster...
154,Original Series,B02,https://jp.pokellector.com/sets/B02-Pokemon-Ju...,Pokemon Jungle Set,https://den-media.pokellector.com/logos/Pokemo...


In [151]:
url_test = 'https://jp.pokellector.com/sets/S8-Fusion-ARTS'

In [152]:
driver.get(url_test)

In [153]:
card_metadata = driver.find_element_by_class_name("setinfo").text
print(card_metadata)

This set introduces new FUSION cards!
Cards
100
+29 Secret
Released
Sep 24th
2021


C:\Users\maxpr\AppData\Local\Temp/ipykernel_20160/3635570042.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  card_metadata = driver.find_element_by_class_name("setinfo").text


In [157]:
card_metadata = driver.find_element_by_class_name("setinfo").text
print(card_metadata)
card_metadata.split("Released")

This set introduces new FUSION cards!
Cards
100
+29 Secret
Released
Sep 24th
2021


C:\Users\maxpr\AppData\Local\Temp/ipykernel_20160/3946651833.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  card_metadata = driver.find_element_by_class_name("setinfo").text


['This set introduces new FUSION cards!\nCards\n100\n+29 Secret\n',
 '\nSep 24th\n2021']

In [155]:
all_cards = driver.find_element_by_class_name('cardlisting').find_elements_by_xpath('//div[@class="card "]')
cards = []
for card in all_cards:
    card_name = card.text
    card_img_ref = card.find_element_by_css_selector('a').find_element_by_css_selector('img').get_attribute('data-src')
    #print(card_name)
    #print(card_img_ref)
    cards.append([card_name, card_img_ref])
df_cards = pd.DataFrame(cards, columns = ['card_name','card_img_url'])

C:\Users\maxpr\AppData\Local\Temp/ipykernel_20160/4231425114.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  all_cards = driver.find_element_by_class_name('cardlisting').find_elements_by_xpath('//div[@class="card "]')
C:\Users\maxpr\AppData\Local\Programs\Python\Python38\lib\site-packages\selenium\webdriver\remote\webelement.py:426: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
C:\Users\maxpr\AppData\Local\Programs\Python\Python38\lib\site-packages\selenium\webdriver\remote\webelement.py:484: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


In [156]:
df_cards

,card_name,card_img_url
0,#1 - Caterpie,https://den-cards.pokellector.com/326/Caterpie...
1,#2 - Metapod,https://den-cards.pokellector.com/326/Metapod....
2,#3 - Butterfree,https://den-cards.pokellector.com/326/Butterfr...
3,#4 - Shroomish,https://den-cards.pokellector.com/326/Shroomis...
4,#5 - Breloom,https://den-cards.pokellector.com/326/Breloom....
...,...,...
124,#125 - Flaaffy,https://den-cards.pokellector.com/326/Flaaffy....
125,#126 - Power Tablet,https://den-cards.pokellector.com/326/Power-Ta...
126,#127 - Training Court,https://den-cards.pokellector.com/326/Training...
127,#128 - Grass Energy,https://den-cards.pokellector.com/326/Grass-En...


In [11]:
from tqdm.notebook import tqdm

def get_all_series(base_url: str) -> None:
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downloads\\chromedriver_win32\\chromedriver.exe" )
    driver.implicitly_wait(10)
    driver.get(base_url)
    series = driver.find_elements_by_class_name('set')
    blocks = driver.find_elements_by_class_name('buttonlisting')
    data_blocks = []
    for idx, block in enumerate(tqdm(blocks[:3])):
    
        #print(ext.text)
        #ext_name = 
        for extension in block.find_elements_by_class_name('button'):
            ext_code = extension.get_attribute('name')
            ext_url = extension.get_attribute('href')
            ext_name = extension.get_attribute('title')
            ext_image_url = extension.find_element_by_css_selector('img').get_attribute('src')
            #print(ext_code)
            #print(ext_url)
            #print(ext_name)
            df_card_ext = one_extension_cards(ext_url)
            df_card_ext['series'] = series[idx].text
            df_card_ext['ext_code'] = ext_code
            df_card_ext['ext_name'] = ext_name
            #data_blocks.append([series[idx].text, ext_code, ext_url, ext_name, ext_image_url])
            df_card_ext.to_csv(ext_name+".csv")
    #df_ext = pd.DataFrame(data_blocks, columns =["serie", "ext_code", "ext_url", "ext_name", "ext_img_url"])


def one_extension_cards(url_extension: str) -> pd.DataFrame:
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downloads\\chromedriver_win32\\chromedriver.exe" )
    driver.get(url_extension)
    card_metadata = driver.find_element_by_class_name("setinfo").text
    print(card_metadata)
    all_cards = driver.find_element_by_class_name('cardlisting').find_elements_by_xpath('//div[@class="card "]')
    cards = []
    for card in tqdm(all_cards):
        card_name = card.text
        card_img_ref = card.find_element_by_css_selector('a').find_element_by_css_selector('img').get_attribute('data-src')
        #print(card_name)
        #print(card_img_ref)
        cards.append([card_name, card_img_ref])
    df_cards = pd.DataFrame(cards, columns = ['card_name','card_img_url'])
    return df_cards

In [12]:
get_all_series(base_url)

C:\Users\maxpr\AppData\Local\Temp/ipykernel_20384/2609232765.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downloads\\chromedriver_win32\\chromedriver.exe" )
C:\Users\maxpr\AppData\Local\Temp/ipykernel_20384/2609232765.py:13: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  series = driver.find_elements_by_class_name('set')
C:\Users\maxpr\AppData\Local\Temp/ipykernel_20384/2609232765.py:14: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  blocks = driver.find_elements_by_class_name('buttonlisting')


  0%|          | 0/3 [00:00<?, ?it/s]

C:\Users\maxpr\AppData\Local\Temp/ipykernel_20384/2609232765.py:44: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downloads\\chromedriver_win32\\chromedriver.exe" )


This set introduced Sparkling (shiny) Pokemon that are limiited to 1 per deck! The set also introduced V & VSTAR versions of the Hisuian starters and featured a cool Pokeball Reverse Holo pattern for some cards!
Cards
67
+5 Secret
Released
Feb 25th
2022


C:\Users\maxpr\AppData\Local\Temp/ipykernel_20384/2609232765.py:46: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  card_metadata = driver.find_element_by_class_name("setinfo").text
C:\Users\maxpr\AppData\Local\Temp/ipykernel_20384/2609232765.py:48: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  all_cards = driver.find_element_by_class_name('cardlisting').find_elements_by_xpath('//div[@class="card "]')


...

A special collection of cards to celebrate the 25th anniversary
Cards
28
+2 Secret
Released
Oct 22nd
2021


  0%|          | 0/38 [00:00<?, ?it/s]

This set introduces new FUSION cards!
Cards
100
+29 Secret
Released
Sep 24th
2021


  0%|          | 0/129 [00:00<?, ?it/s]

KeyboardInterrupt: 